# GenAI v4: Manual Veo Workflow

**3-Step Process:**
1. **Generate** START and END frames locally
2. **Use Veo website** (Frame-to-Video mode)
3. **Replace** scene in original video

## Why This Approach?
- Veo API is complex and has limitations
- Veo website is user-friendly
- **Frame-to-Video** mode gives precise control over transitions
- Results are more consistent

In [ ]:
# Install dependencies if needed
# !pip install torch diffusers transformers opencv-python scenedetect[opencv]

In [ ]:
from GenAI_v4 import FrameGenerator, SceneReplacer

---
## STEP 1: Generate Frames for Veo

This generates:
- **start_frame.png** - Beginning of transition
- **end_frame.png** - End of transition (target background)
- **veo_instructions.txt** - What to input in Veo website

In [ ]:
# Initialize frame generator
generator = FrameGenerator(
    video_dir="data/data_tiktok",
    output_dir="outputs/genai_v4/veo_inputs",
    device="cuda",
)

In [ ]:
# Generate frames for Veo
# Change these values for your video!

VIDEO_ID = "YOUR_VIDEO_ID"  # ← CHANGE THIS
SCENE_INDEX = 6              # ← CHANGE THIS (1-indexed)
ACTION = "increase"          # "increase" (plain bg) or "decrease" (vibrant bg)

inputs = generator.generate_veo_inputs(
    video_id=VIDEO_ID,
    scene_index=SCENE_INDEX,
    action=ACTION,
    transition_style="dramatic",  # "smooth" or "dramatic"
)

In [ ]:
# View the generated frames
from IPython.display import Image, display
from PIL import Image as PILImage
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

axes[0].imshow(PILImage.open(inputs.start_frame_path))
axes[0].set_title("START Frame (upload to Veo)")
axes[0].axis('off')

axes[1].imshow(PILImage.open(inputs.end_frame_path))
axes[1].set_title("END Frame (upload to Veo)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Print Veo instructions
print("=" * 70)
print("  VEO WEBSITE INSTRUCTIONS")
print("=" * 70)
print(f"\n1. Go to: https://labs.google/fx/tools/video-fx")
print(f"\n2. Select: 'Frame to Video' mode")
print(f"\n3. Upload frames:")
print(f"   START: {inputs.start_frame_path}")
print(f"   END:   {inputs.end_frame_path}")
print(f"\n4. Enter this PROMPT:")
print("-" * 70)
print(inputs.prompt)
print("-" * 70)
print(f"\n5. Recommended settings:")
for key, value in inputs.recommended_settings.items():
    print(f"   - {key}: {value}")
print(f"\n6. Generate and download the video")
print(f"\n7. Save as: outputs/genai_v4/veo_inputs/{VIDEO_ID}_scene{SCENE_INDEX}_{ACTION}/veo_output.mp4")
print("=" * 70)

---
## STEP 2: Use Veo Website (Manual)

### Instructions:

1. **Open Veo**: https://labs.google/fx/tools/video-fx

2. **Select Mode**: Choose "Frame to Video"

3. **Upload Frames**:
   - Upload `start_frame.png` as the **first frame**
   - Upload `end_frame.png` as the **last frame**

4. **Enter Prompt**: Copy the prompt printed above

5. **Configure Settings**:
   - **Duration**: Match the scene duration (shown above)
   - **Aspect Ratio**: Match your video (16:9 or 9:16)
   - **Motion**: Minimal / Static camera
   - **Style**: Photorealistic

6. **Generate**: Click generate and wait

7. **Download**: Save as `veo_output.mp4`

8. **Place File**: Put it in the folder shown above

In [ ]:
# Wait for user to complete Veo step
import os

expected_path = f"outputs/genai_v4/veo_inputs/{VIDEO_ID}_scene{SCENE_INDEX}_{ACTION}/veo_output.mp4"

print("Waiting for Veo output...")
print(f"\nPlease save your Veo output to:")
print(f"  {expected_path}")
print(f"\nThen run the next cell.")

In [ ]:
# Check if Veo output exists
if os.path.exists(expected_path):
    print(f"✓ Found Veo output: {expected_path}")
    print("\nReady for Step 3!")
else:
    print(f"✗ Veo output not found: {expected_path}")
    print("\nPlease download from Veo and save to the path above.")

---
## STEP 3: Replace Scene in Original Video

After you've downloaded the Veo output, this step:
1. Detects the exact scene boundaries
2. Resamples Veo video to match frame count
3. Applies color matching for seamless integration
4. Blends at scene boundaries
5. Exports final video

In [ ]:
# Initialize scene replacer
replacer = SceneReplacer(
    video_dir="data/data_tiktok",
    veo_inputs_dir="outputs/genai_v4/veo_inputs",
    output_dir="outputs/genai_v4/final_videos",
)

In [ ]:
# Replace the scene
result = replacer.replace_scene(
    video_id=VIDEO_ID,
    scene_index=SCENE_INDEX,
    action=ACTION,
    blend_frames=5,      # Smooth transition at boundaries
    match_colors=True,   # Match color distribution
)

In [ ]:
# View sample frame from result
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(PILImage.open(result.output_frame_path))
ax.set_title(f"Sample from Final Video (Scene {SCENE_INDEX})")
ax.axis('off')
plt.show()

print(f"\nFinal video saved to: {result.output_video_path}")

---
## Summary

### Files Created

**Step 1 outputs** (in `outputs/genai_v4/veo_inputs/{video}_scene{N}_{action}/`):
- `start_frame.png` - Upload to Veo
- `end_frame.png` - Upload to Veo
- `product_mask.png` - Reference mask
- `veo_instructions.txt` - Full instructions

**Step 2** (manual):
- `veo_output.mp4` - You download this from Veo

**Step 3 outputs** (in `outputs/genai_v4/final_videos/`):
- `{video}_scene{N}_{action}_final.mp4` - Final video
- `{video}_scene{N}_{action}_sample.png` - Sample frame

### Veo Tips
- **Duration**: Match your scene duration
- **Motion**: Keep minimal - the product should stay still
- **Style**: Photorealistic works best
- **Seed**: Note the seed if you want to regenerate

---
## Batch Processing (Multiple Scenes)

In [ ]:
# Process multiple scenes for one video
VIDEO_ID = "YOUR_VIDEO_ID"
SCENES_TO_PROCESS = [1, 3, 5]  # List of scene indices
ACTION = "increase"

# Step 1: Generate all frames
all_inputs = []
for scene_idx in SCENES_TO_PROCESS:
    inputs = generator.generate_veo_inputs(
        video_id=VIDEO_ID,
        scene_index=scene_idx,
        action=ACTION,
    )
    all_inputs.append((scene_idx, inputs))
    print(f"\n" + "="*50)

# Print all paths
print("\n" + "="*70)
print("  FRAMES TO UPLOAD TO VEO")
print("="*70)
for scene_idx, inputs in all_inputs:
    print(f"\nScene {scene_idx}:")
    print(f"  Start: {inputs.start_frame_path}")
    print(f"  End:   {inputs.end_frame_path}")

In [ ]:
# After downloading all Veo outputs, replace all scenes
results = []
for scene_idx, _ in all_inputs:
    try:
        result = replacer.replace_scene(
            video_id=VIDEO_ID,
            scene_index=scene_idx,
            action=ACTION,
        )
        results.append((scene_idx, result))
        print(f"✓ Scene {scene_idx} replaced")
    except FileNotFoundError as e:
        print(f"✗ Scene {scene_idx}: {e}")